In [1]:
import os
import sys
import math
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from datetime import datetime

In [2]:
base_path = os.environ['HOME'] + '/cifar100/AdaNet'
if os.path.isdir(base_path):
  pass
else:
  os.mkdir(base_path)

In [3]:
data = tf.keras.datasets.cifar100.load_data()

train = data[0]
test = data[1]

train_image, train_label = train[0].astype(np.float32), train[1]
test_image, test_label= test[0].astype(np.float32), test[1]

train_label = np.reshape(train_label, newshape=[-1])
test_label = np.reshape(test_label, newshape=[-1])

In [4]:
# Generate Dataset obj   
dataset_obj = tf.data.Dataset.from_tensors(
    {'image': train_image, 'label': train_label})
dataset_obj = dataset_obj.shuffle(50000)
dataset_obj = dataset_obj.unbatch()

# split train-validation dataset
train_dataset = dataset_obj.take(40000)
val_dataset = dataset_obj.skip(40000).take(10000)

test_dataset = tf.data.Dataset.from_tensors(
  {'image': test_image, 'label': test_label})
test_dataset = test_dataset.shuffle(10000).unbatch()

def _preprocessing(dataset, train_mode):
  """
  While train steps, image will be padded random crop and filped(horizontaly)
  And entire steps, per-pixel mean subtracted will be required.
  Args:
    dataset: 'tf.data.Dataset'
    train_mode: 'bool'
  Returns:
    'tf.data.Dataset'
  """
  if train_mode:
    image = dataset['image']
    pad = tf.constant([[2, 2], [2, 2], [0, 0]])
    image = tf.pad(
      tensor=image, paddings=pad)
    image = tf.image.random_crop(
      value=image, size=[32, 32, 3])
    image = tf.image.random_flip_left_right(image=image)
  else:
    image = dataset['image']
    
  image = tf.math.subtract(
    x=image,
    y=tf.reshape(
      tf.math.reduce_mean(image, axis=2), shape=[32, 32, 1]))
  label = dataset['label']
  return (image, label)

train_dataset = train_dataset.map(
  lambda x: _preprocessing(x, train_mode=True))
val_dataset = val_dataset.map(
  lambda x: _preprocessing(x, train_mode=False))
test_dataset = test_dataset.map(
  lambda x: _preprocessing(x, train_mode=False))


train_dataset = train_dataset.repeat()
val_dataset = val_dataset.repeat()

In [5]:
# WEAK LEARNER 
def residual_block_tensor(x):
  identity_x = x
  identity_x = keras.layers.Conv2D(
    filters=256, kernel_size=[1, 1], strides=1, padding='same')(identity_x)
  identity_x = keras.layers.BatchNormalization()(identity_x)
  identity_x = keras.layers.ReLU()(identity_x)
  
  x = keras.layers.Conv2D(
    filters=64, kernel_size=[1, 1], strides=1, padding='same')(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.ReLU()(x)

  x = keras.layers.Conv2D(
    filters=64, kernel_size=[3, 3], strides=1, padding='same')(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.ReLU()(x)
  
  x = keras.layers.Conv2D(
    filters=256, kernel_size=[1, 1], strides=1, padding='same')(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.ReLU()(x)
  
  x = keras.layers.Add()([x, identity_x])
  x = keras.layers.ReLU()(x)
  return x


# residual block custom layer

class ResidualBlock(keras.layers.Layer):
  def __init__(self, **kwargs):
    super(ResidualBlock, self).__init__()
    self.conv_idt = keras.layers.Conv2D(
      filters=3, kernel_size=[1, 1], strides=1, padding='same')
    self.bn_idt = keras.layers.BatchNormalization()
    self.relu_idt = keras.layers.ReLU()
    
    self.conv_1 = keras.layers.Conv2D(
      filters=64, kernel_size=[1, 1], strides=1, padding='same')
    self.bn_1 = keras.layers.BatchNormalization()
    self.relu_1 = keras.layers.ReLU()
    
    self.conv_2 = keras.layers.Conv2D(
      filters=64, kernel_size=[1, 1], strides=1, padding='same')
    self.bn_2 = keras.layers.BatchNormalization()
    self.relu_2 = keras.layers.ReLU()
    
    self.conv_3 = keras.layers.Conv2D(
      filters=3, kernel_size=[1, 1], strides=1, padding='same')
    self.bn_3 = keras.layers.BatchNormalization()
    self.relu_3 = keras.layers.ReLU()
    
    self.add_last = keras.layers.Add()
    self.relu_last = keras.layers.ReLU()
    
  def call(self, inputs):
    idt_x = self.conv_idt(inputs)
    idt_x = self.bn_idt(idt_x)
    idt_x = self.relu_idt(idt_x)
    x = self.conv_1(inputs)
    x = self.bn_1(x)
    x = self.relu_1(x)
    x = self.conv_2(x)
    x = self.bn_2(x)
    x = self.relu_2(x)
    x = self.conv_3(x)
    x = self.bn_3(x)
    x = self.relu_3(x)
    x = self.add_last([x, idt_x])
    return self.relu_last(x)
    